# Portfolio Stress Test - Delta Calculations (Clean Version)

This notebook calculates portfolio delta exposure by:
1. Loading preprocessed position data
2. **Filtering options to only include Source = 'options-data-range'**
3. Calculating delta for each position type
4. Creating clean breakdown by token (Spot/Future/Options)
5. Running stress test scenarios

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display options for better output - no scientific notation
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.float_format', '{:.8f}'.format)
np.set_printoptions(suppress=True)

## Step 1: Load and Filter Data

In [3]:
# Load the preprocessed positions data
df = pd.read_csv('processed_positions.csv')

original_count = len(df)
print(f"Loaded {original_count:,} positions")
print(f"Unique tokens: {df['Token'].nunique():,}")
print(f"Position types: {list(df['Position_Type'].unique())}")

# Check options by source
option_mask = df['Position_Type'].str.upper() == 'OPTION'
if option_mask.any():
    print(f"\nOption positions by source:")
    option_sources = df[option_mask]['Source'].value_counts()
    for source, count in option_sources.items():
        print(f"  {source}: {count:,} positions")

# FILTER: Keep only options with source = 'options-data-range' OR non-option positions
filtered_mask = (
    (df['Position_Type'].str.upper() != 'OPTION') |  # Keep all non-options
    (df['Source'] != 'written-off')  # Keep only options-data-range options + deribit
)

df = df[filtered_mask].copy()
filtered_count = len(df)
dropped_count = original_count - filtered_count

print(f"\n✅ FILTERED DATA:")
print(f"- Kept: {filtered_count:,} positions")
print(f"- Dropped: {dropped_count:,} positions (non-options-data-range options)")
print(f"- Spot positions: {(df['Position_Type'].str.upper() == 'SPOT').sum():,}")
print(f"- Future positions: {(df['Position_Type'].str.upper() == 'FUTURE').sum():,}")
print(f"- Option positions (options-data-range only): {(df['Position_Type'].str.upper() == 'OPTION').sum():,}")

Loaded 192 positions
Unique tokens: 23
Position types: ['Spot', 'Option', 'Future']

Option positions by source:
  written-off: 114 positions
  options-data-range: 31 positions
  deribit: 2 positions

✅ FILTERED DATA:
- Kept: 78 positions
- Dropped: 114 positions (non-options-data-range options)
- Spot positions: 18
- Future positions: 27
- Option positions (options-data-range only): 33


## Step 2: Black-Scholes Functions

In [4]:
def black_scholes_delta(S, K, T, r, sigma, option_type='call', token=''):
    """
    Calculate option delta using Black-Scholes model
    Interest rate = 0, Uses volatility from data
    For ETH options, subtract premium from delta
    """
    debug_eth = token.upper() == 'ETH'
    
    if debug_eth:
        print(f"\n--- BLACK-SCHOLES DEBUG for {token} ---")
        print(f"Inputs: S={S}, K={K}, T={T}, r={r}, sigma={sigma}, type={option_type}")
    
    if T <= 0:
        # Option expired
        if option_type.lower() == 'call':
            result = 1.0 if S > K else 0.0
        else:  # put
            result = -1.0 if S < K else 0.0
        if debug_eth:
            print(f"Option expired (T={T}), returning: {result}")
        return result
    
    if sigma <= 0 or S <= 0 or K <= 0:
        if debug_eth:
            print(f"Invalid inputs - sigma={sigma}, S={S}, K={K}, returning 0.0")
        return 0.0
    
    try:
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        
        if debug_eth:
            print(f"d1 calculation: ln({S}/{K}) + ({r} + 0.5*{sigma}²)*{T} / ({sigma}*√{T}) = {d1}")
        
        if option_type.lower() == 'call':
            delta = norm.cdf(d1)
            if debug_eth:
                print(f"Call delta: N(d1) = N({d1}) = {delta}")
        else:  # put
            delta = norm.cdf(d1) - 1.0
            if debug_eth:
                print(f"Put delta: N(d1) - 1 = N({d1}) - 1 = {delta}")
        
        # Store ETH premium info for later spot adjustment (don't adjust option delta)
        eth_premium_eth = 0.0
        if token.upper() == 'ETH':
            premium_usd = black_scholes_premium(S, K, T, r, sigma, option_type)
            eth_premium_eth = premium_usd / S  # Convert premium to ETH terms
            if debug_eth:
                print(f"ETH premium info (will adjust spot, not option delta):")
                print(f"  Premium (USD): {premium_usd}")
                print(f"  Premium (ETH): {eth_premium_eth}")
                print(f"  Option delta (unadjusted): {delta}")
        
        if debug_eth:
            print(f"Final option delta: {delta}")
            print("--- END DEBUG ---\n")
            
        return delta, eth_premium_eth if token.upper() == 'ETH' else 0.0
    except Exception as e:
        if debug_eth:
            print(f"Exception in calculation: {e}")
        return 0.0

def calculate_time_to_expiry(expiry_str, current_date=None):
    """Calculate time to expiry in years from expiry string"""
    if pd.isna(expiry_str) or expiry_str == '':
        return 0.0
    
    if current_date is None:
        current_date = datetime.now()
    
    try:
        expiry_str = str(expiry_str).strip()
        
        # If expiry only has date (no time), append 10:00 for 10:00 UTC
        if ':' not in expiry_str and len(expiry_str.split('/')) >= 2:
            expiry_str = expiry_str + ' 10:00'
            print(f"DEBUG: Added default time to expiry: '{expiry_str}'")
        
        # Comprehensive date formats
        date_formats = [
            '%d/%m/%Y %H:%M',     # 02/09/2025 10:00
            '%d/%m/%Y',           # 02/09/2025 (without time)
            '%Y-%m-%d %H:%M',     # 2025-09-02 10:00
            '%Y-%m-%d',           # 2025-09-02 (without time)
            '%d%b%y %H:%M',       # 26Sep25 09:00 (deribit format)
            '%d%B%y %H:%M',       # 26September25 09:00 (full month)
            '%d/%b/%Y %H:%M',     # 02/Sep/2025 10:00
            '%d/%B/%Y %H:%M',     # 02/September/2025 10:00
            '%dSep%y %H:%M',      # Specific months
            '%dOct%y %H:%M', 
            '%dNov%y %H:%M', 
            '%dDec%y %H:%M',
            '%dJan%y %H:%M', 
            '%dFeb%y %H:%M', 
            '%dMar%y %H:%M',
            '%dApr%y %H:%M', 
            '%dMay%y %H:%M', 
            '%dJun%y %H:%M', 
            '%dJul%y %H:%M',
            '%dAug%y %H:%M'
        ]
        
        # Debug print for important dates
        if any(month in expiry_str for month in ['Sep', 'Oct', 'Nov', 'Dec', '2025', '2026']) or 'ETH' in str(current_date):
            print(f"DEBUG: Parsing expiry date: '{expiry_str}'")
        
        for fmt in date_formats:
            try:
                expiry_date = datetime.strptime(expiry_str, fmt)
                time_diff = (expiry_date - current_date).total_seconds()
                years = time_diff / (365.25 * 24 * 3600)
                
                # Debug for important dates
                if any(month in expiry_str for month in ['Sep', 'Oct', 'Nov', 'Dec', '2025', '2026']):
                    print(f"DEBUG: Successfully parsed '{expiry_str}' with format '{fmt}'")
                    print(f"DEBUG: Expiry date: {expiry_date}")
                    print(f"DEBUG: Current date: {current_date}")
                    print(f"DEBUG: Time to expiry (years): {years}")
                
                return max(0.0, years)
            except:
                continue
        
        print(f"WARNING: Could not parse expiry date: '{expiry_str}' with any format")
        return 0.0
        
    except Exception as e:
        print(f"ERROR parsing expiry {expiry_str}: {e}")
        return 0.0

def black_scholes_premium(S, K, T, r, sigma, option_type='call'):
    """
    Calculate option premium using Black-Scholes model
    """
    if T <= 0:
        # Option expired - intrinsic value only
        if option_type.lower() == 'call':
            return max(0, S - K)
        else:  # put
            return max(0, K - S)
    
    if sigma <= 0 or S <= 0 or K <= 0:
        return 0.0
    
    try:
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type.lower() == 'call':
            premium = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        else:  # put
            premium = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
            
        return max(0, premium)
    except:
        return 0.0

# Test functions
print("Testing Black-Scholes (r=0):")
test_call_result = black_scholes_delta(S=100, K=100, T=0.25, r=0.0, sigma=0.2, option_type='call')
test_put_result = black_scholes_delta(S=100, K=100, T=0.25, r=0.0, sigma=0.2, option_type='put')

# Handle tuple format for non-ETH options (returns single delta value)
if isinstance(test_call_result, tuple):
    test_call_delta = test_call_result[0]
else:
    test_call_delta = test_call_result

if isinstance(test_put_result, tuple):
    test_put_delta = test_put_result[0]
else:
    test_put_delta = test_put_result

print(f"ATM Call delta: {test_call_delta:.4f}")
print(f"ATM Put delta:  {test_put_delta:.4f}")

Testing Black-Scholes (r=0):
ATM Call delta: 0.5199
ATM Put delta:  -0.4801


## Step 3: Calculate Delta for All Positions

In [5]:
# Initialize delta columns
df['Delta'] = 0.0
df['Time_to_Expiry'] = 0.0
df['ETH_Premium_Adjustment'] = 0.0  # Track ETH premium adjustments

print("Calculating delta for filtered data...")

# Track total ETH premium to subtract from ETH spot positions
total_eth_premium_adjustment = 0.0

# 1. Spot positions: Delta = 1.0 (will adjust ETH later)
spot_mask = df['Position_Type'].str.upper() == 'SPOT'
df.loc[spot_mask, 'Delta'] = 1.0
print(f"Spot positions: {spot_mask.sum():,} (Delta = 1.0, ETH will be adjusted for option premiums)")

# 2. Future positions: Delta = 1.0
future_mask = df['Position_Type'].str.upper() == 'FUTURE'
df.loc[future_mask, 'Delta'] = 1.0
print(f"Future positions: {future_mask.sum():,} (Delta = 1.0)")

# 3. Option positions: Calculate using Black-Scholes
option_mask = df['Position_Type'].str.upper() == 'OPTION'
option_count = option_mask.sum()
print(f"Option positions: {option_count:,} (Black-Scholes, ETH premium adjustment applied to spot)")

if option_count > 0:
    # Verify option sources
    option_sources = df[option_mask]['Source'].unique()
    print(f"Option sources in filtered data: {list(option_sources)}")
    
    # Calculate time to expiry
    df.loc[option_mask, 'Time_to_Expiry'] = df.loc[option_mask, 'Expiry'].apply(calculate_time_to_expiry)
    
    calculated_deltas = 0
    for idx, row in df[option_mask].iterrows():
        S = row['Current_Price']
        K = row['Strike']
        T = row['Time_to_Expiry']
        r = 0.0  # Interest rate = 0
        sigma = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
        option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
        token = row['Token']
        
        # DEBUG: Print ETH option details
        if token.upper() == 'ETH':
            print(f"\n=== ETH OPTION DEBUG ===")
            print(f"Index: {idx}")
            print(f"Token: {token}")
            print(f"Source: {row['Source']}")
            print(f"Expiry: {row['Expiry']}")
            print(f"Strike: {K}")
            print(f"Size: {row['Size']}")
            print(f"Option_Type: {row['Option_Type']}")
            print(f"Current_Price: {S}")
            print(f"Volatility: {row['Volatility']}")
            print(f"Time_to_Expiry: {T}")
            print(f"Cleaned option_type: {option_type}")
        
        # Clean option type
        if 'call' in option_type:
            option_type = 'call'
        elif 'put' in option_type:
            option_type = 'put'
        else:
            option_type = 'call'
        
        if pd.notna(S) and pd.notna(K) and S > 0 and K > 0:
            delta_result = black_scholes_delta(S, K, T, r, sigma, option_type, token)
            
            # Handle new return format (delta, eth_premium_adjustment)
            if isinstance(delta_result, tuple):
                delta, eth_premium_adjustment = delta_result
            else:
                delta = delta_result
                eth_premium_adjustment = 0.0
            
            # DEBUG: Print ETH delta calculation details
            if token.upper() == 'ETH':
                print(f"Option delta (unadjusted): {delta:.6f}")
                print(f"ETH premium adjustment: {eth_premium_adjustment:.6f}")
                print("========================\n")
                
                # Accumulate ETH premium adjustments
                total_eth_premium_adjustment += eth_premium_adjustment * row['Size']
            
            df.at[idx, 'Delta'] = delta
            df.at[idx, 'ETH_Premium_Adjustment'] = eth_premium_adjustment
            calculated_deltas += 1
        else:
            # DEBUG: Print why ETH option was skipped
            if token.upper() == 'ETH':
                print(f"ETH option SKIPPED - S: {S}, K: {K}, S>0: {S>0}, K>0: {K>0}")
    
    print(f"Calculated delta for {calculated_deltas:,} options")
    
    # Apply ETH premium adjustment to ETH spot positions
    eth_spot_mask = (df['Position_Type'].str.upper() == 'SPOT') & (df['Token'].str.upper() == 'ETH')
    if eth_spot_mask.any() and total_eth_premium_adjustment != 0:
        print(f"\nApplying ETH premium adjustment to spot positions:")
        print(f"Total ETH premium to subtract from spot: {total_eth_premium_adjustment:.6f} ETH")
        
        # Simple adjustment: subtract total premium from all ETH spot positions
        for idx, row in df[eth_spot_mask].iterrows():
            original_delta = df.at[idx, 'Delta']
            # Calculate per-unit adjustment for this position
            adjustment_per_unit = total_eth_premium_adjustment / row['Size']
            new_delta = original_delta - adjustment_per_unit
            
            df.at[idx, 'ETH_Premium_Adjustment'] = adjustment_per_unit
            df.at[idx, 'Delta'] = new_delta
            
            print(f"  ETH Spot position {idx}: Size={row['Size']:.0f}")
            print(f"    Premium adjustment per unit: {adjustment_per_unit:.6f} ETH")
            print(f"    Original delta: {original_delta:.6f}")
            print(f"    New delta: {new_delta:.6f}")
    elif total_eth_premium_adjustment != 0:
        print(f"\nWARNING: ETH premium adjustment of {total_eth_premium_adjustment:.6f} ETH calculated but no ETH spot positions found!")

# Calculate delta exposure
df['Delta_Exposure'] = df['Size'] * df['Delta']
df['Delta_Exposure_USD'] = df['Delta_Exposure'] * df['Current_Price']

print(f"\n✅ Delta calculation complete for {len(df):,} positions")
print(f"Total portfolio delta exposure: ${df['Delta_Exposure_USD'].sum():,.2f}")
if total_eth_premium_adjustment != 0:
    print(f"ETH premium adjustment applied: {total_eth_premium_adjustment:.6f} ETH (${total_eth_premium_adjustment * df[df['Token']=='ETH']['Current_Price'].iloc[0]:,.2f})")

Calculating delta for filtered data...
Spot positions: 18 (Delta = 1.0, ETH will be adjusted for option premiums)
Future positions: 27 (Delta = 1.0)
Option positions: 33 (Black-Scholes, ETH premium adjustment applied to spot)
Option sources in filtered data: ['options-data-range', 'deribit']
DEBUG: Added default time to expiry: '16/09/2025 10:00'
DEBUG: Parsing expiry date: '16/09/2025 10:00'
DEBUG: Successfully parsed '16/09/2025 10:00' with format '%d/%m/%Y %H:%M'
DEBUG: Expiry date: 2025-09-16 10:00:00
DEBUG: Current date: 2025-09-08 09:18:16.349931
DEBUG: Time to expiry (years): 0.02198214218029888
DEBUG: Added default time to expiry: '16/09/2025 10:00'
DEBUG: Parsing expiry date: '16/09/2025 10:00'
DEBUG: Successfully parsed '16/09/2025 10:00' with format '%d/%m/%Y %H:%M'
DEBUG: Expiry date: 2025-09-16 10:00:00
DEBUG: Current date: 2025-09-08 09:18:16.349931
DEBUG: Time to expiry (years): 0.02198214218029888
DEBUG: Added default time to expiry: '16/09/2025 10:00'
DEBUG: Parsing ex

In [6]:
# Display option expiry dates sorted from closest to furthest
print("=" * 80)
print("OPTION EXPIRY DATES (Closest to Furthest)")
print("=" * 80)

option_positions = df[df['Position_Type'].str.upper() == 'OPTION'].copy()

if len(option_positions) > 0:
    # Sort by time to expiry (closest first)
    option_positions_sorted = option_positions.sort_values('Time_to_Expiry')
    
    print(f"{'Token':<8} {'Expiry Date':<20} {'Time to Expiry':<15} {'Strike':<12} {'Type':<6} {'Size':<12}")
    print("-" * 80)
    
    for _, row in option_positions_sorted.iterrows():
        token = row['Token']
        expiry = row['Expiry']
        time_to_expiry = row['Time_to_Expiry']
        strike = row['Strike'] if pd.notna(row['Strike']) else 0
        opt_type = str(row['Option_Type'])[:4] if pd.notna(row['Option_Type']) else 'N/A'
        size = row['Size']
        
        # Format time to expiry
        if time_to_expiry < 0.003:  # Less than ~1 day
            time_str = f"{time_to_expiry*365:.1f} days"
        elif time_to_expiry < 0.1:  # Less than ~36 days
            time_str = f"{time_to_expiry*365:.0f} days"
        else:
            time_str = f"{time_to_expiry:.3f} years"
        
        print(f"{token:<8} {expiry:<20} {time_str:<15} {strike:>11.2f} {opt_type:<6} {size:>11,.0f}")
    
    print(f"\nTotal option positions: {len(option_positions):,}")
    print(f"Expiry range: {option_positions_sorted['Time_to_Expiry'].min():.4f} to {option_positions_sorted['Time_to_Expiry'].max():.4f} years")
else:
    print("No option positions found.")

OPTION EXPIRY DATES (Closest to Furthest)
Token    Expiry Date          Time to Expiry  Strike       Type   Size        
--------------------------------------------------------------------------------
CFX      10/09/2025           2 days                 0.11 CALL     1,000,000
CFX      10/09/2025           2 days                 0.14 CALL     1,000,000
CFX      10/09/2025           2 days                 0.19 CALL     1,000,000
CFX      10/09/2025           2 days                 0.16 CALL     1,000,000
OXT      16/09/2025           8 days                 0.04 Put      5,000,000
OXT      16/09/2025           8 days                 0.06 Call     5,000,000
OXT      16/09/2025           8 days                 0.05 Call     5,000,000
OXT      16/09/2025           8 days                 0.06 Call     5,000,000
OXT      16/09/2025           8 days                 0.05 Put      5,000,000
SAFE     17/09/2025           9 days                 0.40 PUT        308,040
SAFE     17/09/2025         

## Step 4: Clean Delta Breakdown by Token

In [7]:
print("=" * 100)
print("PORTFOLIO DELTA EXPOSURE BY TOKEN (CLEAN VIEW)")
print("=" * 100)

# Create summary by token
summary_data = []

# Get tokens sorted by absolute exposure
token_totals = df.groupby('Token')['Delta_Exposure_USD'].sum().abs().sort_values(ascending=False)

for token in token_totals.index:
    token_data = df[df['Token'] == token]
    
    # Spot delta
    spot_data = token_data[token_data['Position_Type'].str.upper() == 'SPOT']
    spot_delta = spot_data['Delta_Exposure'].sum() if len(spot_data) > 0 else 0.0
    spot_usd = spot_data['Delta_Exposure_USD'].sum() if len(spot_data) > 0 else 0.0
    
    # Future delta
    future_data = token_data[token_data['Position_Type'].str.upper() == 'FUTURE']
    future_delta = future_data['Delta_Exposure'].sum() if len(future_data) > 0 else 0.0
    future_usd = future_data['Delta_Exposure_USD'].sum() if len(future_data) > 0 else 0.0
    
    # Options delta
    option_data = token_data[token_data['Position_Type'].str.upper() == 'OPTION']
    option_delta = option_data['Delta_Exposure'].sum() if len(option_data) > 0 else 0.0
    option_usd = option_data['Delta_Exposure_USD'].sum() if len(option_data) > 0 else 0.0
    
    # Totals
    total_delta = spot_delta + future_delta + option_delta
    total_usd = spot_usd + future_usd + option_usd
    current_price = token_data['Current_Price'].iloc[0]
    
    summary_data.append({
        'Token': token,
        'Spot_Delta': spot_delta,
        'Future_Delta': future_delta,
        'Option_Delta': option_delta,
        'Total_Delta': total_delta,
        'Spot_USD': spot_usd,
        'Future_USD': future_usd,
        'Option_USD': option_usd,
        'Total_USD': total_usd,
        'Price': current_price
    })

# Display clean table
print(f"{'Token':<8} {'Spot Δ':<12} {'Future Δ':<12} {'Option Δ':<12} {'Total Δ':<12} {'Total USD':<15} {'Price':<12}")
print("=" * 100)

for row in summary_data:
    print(f"{row['Token']:<8} {row['Spot_Delta']:>11,.1f} {row['Future_Delta']:>11,.1f} {row['Option_Delta']:>11,.1f} {row['Total_Delta']:>11,.1f} {row['Total_USD']:>12,.0f} ${row['Price']:>10,.6f}")

# Summary totals
print("=" * 100)
total_spot_usd = sum(row['Spot_USD'] for row in summary_data)
total_future_usd = sum(row['Future_USD'] for row in summary_data)
total_option_usd = sum(row['Option_USD'] for row in summary_data)
portfolio_total = sum(row['Total_USD'] for row in summary_data)

print(f"Portfolio Summary:")
print(f"  Spot Exposure:    ${total_spot_usd:>15,.0f}")
print(f"  Future Exposure:  ${total_future_usd:>15,.0f}")
print(f"  Option Exposure:  ${total_option_usd:>15,.0f}")
print(f"  TOTAL EXPOSURE:   ${portfolio_total:>15,.0f}")

# Show options detail if any exist
option_positions = df[df['Position_Type'].str.upper() == 'OPTION']
if len(option_positions) > 0:
    print("\n" + "=" * 80)
    print("OPTIONS DETAIL")
    print("=" * 80)
    print(f"{'Token':<8} {'Strike':<10} {'Type':<6} {'Size':<12} {'Delta':<8} {'Δ Exposure':<12} {'USD Value':<12}")
    print("-" * 80)
    
    for _, row in option_positions.iterrows():
        token = row['Token']
        strike = row['Strike'] if pd.notna(row['Strike']) else 0
        opt_type = str(row['Option_Type'])[:4] if pd.notna(row['Option_Type']) else 'N/A'
        size = row['Size']
        delta = row['Delta']
        delta_exp = row['Delta_Exposure']
        usd_val = row['Delta_Exposure_USD']
        
        print(f"{token:<8} {strike:>9.4f} {opt_type:<6} {size:>11,.0f} {delta:>7.3f} {delta_exp:>11,.0f} ${usd_val:>10,.0f}")

PORTFOLIO DELTA EXPOSURE BY TOKEN (CLEAN VIEW)
Token    Spot Δ       Future Δ     Option Δ     Total Δ      Total USD       Price       
ETH            184.5      -242.6       132.4        74.3      327,112 $4,404.074004
ICP       -619,813.0   414,944.8   156,866.9   -48,001.3     -230,386 $  4.799583
SOL              0.0       993.9         0.0       993.9      205,071 $206.340000
BTC              7.2        -8.5         0.0        -1.3     -149,726 $112,194.995900
XRP              0.0    25,445.5         0.0    25,445.5       72,357 $  2.843600
SAFE       -47,430.5   122,234.3         0.0    74,803.8       31,588 $  0.422280
TOWNS            0.0 1,352,115.4         0.0 1,352,115.4       31,504 $  0.023300
SUI              0.0     7,992.2         0.0     7,992.2       27,066 $  3.386500
NEAR      -383,392.3   251,451.2   143,061.3    11,120.3       26,938 $  2.422444
OXT      -3,153,056.7         0.0 2,721,885.5  -431,171.1      -23,051 $  0.053463
CSPR      -660,152.9         0.0 3,0

## Step 5: Stress Test - 5% Price Moves

In [8]:
# Stress Test: 5% Price Moves Up and Down
print("=" * 100)
print("STRESS TEST: 10% PRICE MOVES")
print("=" * 100)

stress_scenarios = [
    {"name": "10% UP", "multiplier": 1.1},
    {"name": "10% DOWN", "multiplier": 0.9}
]

for scenario in stress_scenarios:
    print(f"\n{scenario['name']} SCENARIO:")
    print("-" * 60)
    print(f"{'Token':<8} {'Spot PnL':<12} {'Future PnL':<12} {'Option PnL':<12} {'Total PnL':<12}")
    print("-" * 60)
    
    total_spot_pnl = 0.0
    total_future_pnl = 0.0
    total_option_pnl = 0.0
    
    # Group by token for cleaner output
    for token in df['Token'].unique():
        token_data = df[df['Token'] == token]
        current_price = token_data['Current_Price'].iloc[0]
        stressed_price = current_price * scenario['multiplier']
        price_change = stressed_price - current_price
        
        spot_pnl = 0.0
        future_pnl = 0.0
        option_pnl = 0.0
        
        # Spot PnL - use adjusted delta exposure (accounts for ETH premium adjustments)
        spot_data = token_data[token_data['Position_Type'].str.upper() == 'SPOT']
        if len(spot_data) > 0:
            total_spot_delta_exposure = spot_data['Delta_Exposure'].sum()
            spot_pnl = total_spot_delta_exposure * price_change
        
        # Future PnL - use adjusted delta exposure 
        future_data = token_data[token_data['Position_Type'].str.upper() == 'FUTURE']
        if len(future_data) > 0:
            total_future_delta_exposure = future_data['Delta_Exposure'].sum()
            future_pnl = total_future_delta_exposure * price_change
        
        # Options PnL (recalculate option prices)
        option_data = token_data[token_data['Position_Type'].str.upper() == 'OPTION']
        if len(option_data) > 0:
            for _, row in option_data.iterrows():
                size = row['Size']
                
                # Current option price
                S_current = current_price
                K = row['Strike']
                T = row['Time_to_Expiry']
                r = 0.0
                sigma = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
                option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
                
                if 'call' in option_type:
                    option_type = 'call'
                elif 'put' in option_type:
                    option_type = 'put'
                else:
                    option_type = 'call'
                
                if pd.notna(S_current) and pd.notna(K) and S_current > 0 and K > 0:
                    current_option_price = black_scholes_premium(S_current, K, T, r, sigma, option_type)
                    
                    # Stressed option price
                    stressed_option_price = black_scholes_premium(stressed_price, K, T, r, sigma, option_type)
                    
                    # Option PnL
                    single_option_pnl = size * (stressed_option_price - current_option_price)
                    option_pnl += single_option_pnl
        
        total_pnl = spot_pnl + future_pnl + option_pnl
        
        # Only show tokens with significant PnL
        if abs(total_pnl) > 10:
            print(f"{token:<8} ${spot_pnl:>10,.0f} ${future_pnl:>11,.0f} ${option_pnl:>11,.0f} ${total_pnl:>11,.0f}")
        
        total_spot_pnl += spot_pnl
        total_future_pnl += future_pnl
        total_option_pnl += option_pnl
    
    print("-" * 60)
    print(f"{'TOTAL':<8} ${total_spot_pnl:>10,.0f} ${total_future_pnl:>11,.0f} ${total_option_pnl:>11,.0f} ${total_spot_pnl + total_future_pnl + total_option_pnl:>11,.0f}")

print("\n" + "=" * 100)

STRESS TEST: 10% PRICE MOVES

10% UP SCENARIO:
------------------------------------------------------------
Token    Spot PnL     Future PnL   Option PnL   Total PnL   
------------------------------------------------------------
OXT      $   -16,857 $          0 $     29,142 $     12,285
FET      $   -29,899 $     31,809 $          0 $      1,911
ICP      $  -297,484 $    199,156 $     90,747 $     -7,581
ZEN      $       436 $          0 $          0 $        436
CSPR     $      -640 $          0 $      3,267 $      2,627
ACX      $      -209 $        972 $      1,375 $      2,138
IOST     $   -28,086 $      4,727 $     27,874 $      4,514
SAFE     $    -2,003 $      5,162 $          0 $      3,159
NEAR     $   -92,875 $     60,913 $     39,856 $      7,894
CELO     $   -10,397 $      3,643 $      8,989 $      2,235
CFX      $   -46,268 $     -5,512 $     55,169 $      3,390
VET      $   -10,965 $      1,187 $     11,185 $      1,407
MINA     $   -10,402 $          0 $     11,196 $  

## Step 6: PnL Visualization

In [9]:
# Debug ETH Options - Detailed Tabular Analysis
eth_options = df[(df['Token'] == 'ETH') & (df['Position_Type'].str.upper() == 'OPTION')].copy()

if len(eth_options) > 0:
    print("=" * 120)
    print("ETH OPTIONS DETAILED DEBUG - PnL ANALYSIS")
    print("=" * 120)
    
    current_eth_price = eth_options['Current_Price'].iloc[0]
    print(f"Current ETH Price: ${current_eth_price:,.2f}")
    print(f"Number of ETH Options: {len(eth_options)}")
    print()
    
    # Create detailed breakdown for each option position
    for idx, (_, row) in enumerate(eth_options.iterrows()):
        strike = row['Strike']
        size = row['Size']
        expiry = row['Expiry']
        time_to_expiry = row['Time_to_Expiry']
        option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
        volatility = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
        
        # Clean option type
        if 'call' in option_type:
            option_type = 'call'
        elif 'put' in option_type:
            option_type = 'put'
        else:
            option_type = 'call'
        
        print(f"\n--- ETH OPTION #{idx+1} ---")
        print(f"Strike: ${strike:,.2f}")
        print(f"Size: {size:,.0f}")
        print(f"Expiry: {expiry}")
        print(f"Time to Expiry: {time_to_expiry:.4f} years")
        print(f"Type: {option_type.upper()}")
        print(f"Volatility: {volatility:.2%}")
        
        # Calculate current option price for PnL reference
        current_option_price = black_scholes_premium(current_eth_price, strike, time_to_expiry, 0.0, volatility, option_type)
        current_total_value = current_option_price * size
        
        print(f"Current Premium per Contract: ${current_option_price:,.2f}")
        print(f"Current Total Position Value: ${current_total_value:,.0f} (${current_option_price:,.2f} × {size:,.0f})")
        print()
        
        # Calculate option prices and PnL at every 1% move from -10% to +10%
        price_moves = list(range(-10, 11, 1))  # -10% to +10% in 1% steps
        
        print(f"{'Price Move':<12} {'ETH Price':<12} {'Premium':<12} {'Total Value':<15} {'Position PnL':<15}")
        print(f"{'(%)':<12} {'($)':<12} {'($)':<12} {'(Premium*Size)':<15} {'($)':<15}")
        print("-" * 80)
        
        for pct_change in price_moves:
            multiplier = 1 + (pct_change / 100)
            stressed_eth_price = current_eth_price * multiplier
            
            # Calculate stressed option price (premium per contract)
            stressed_option_price = black_scholes_premium(stressed_eth_price, strike, time_to_expiry, 0.0, volatility, option_type)
            
            # Calculate total position value (premium * size)
            total_position_value = stressed_option_price * size
            
            # Calculate PnL for this position
            option_price_change = stressed_option_price - current_option_price
            position_pnl = size * option_price_change
            
            print(f"{pct_change:>+3d}%       ${stressed_eth_price:>9,.0f}   ${stressed_option_price:>10,.2f}   ${total_position_value:>13,.0f}   ${position_pnl:>13,.0f}")
        
        print()
    
    # Summary table showing total ETH options PnL at each price level
    print("\n" + "=" * 80)
    print("ETH OPTIONS PORTFOLIO SUMMARY - TOTAL PnL BY PRICE MOVE")
    print("=" * 80)
    
    print(f"{'Price Move':<12} {'ETH Price':<12} {'Total Options PnL':<20} {'Individual PnL Contributions'}")
    print(f"{'(%)':<12} {'($)':<12} {'($)':<20} {'(Position #: PnL)'}")
    print("-" * 100)
    
    for pct_change in price_moves:
        multiplier = 1 + (pct_change / 100)
        stressed_eth_price = current_eth_price * multiplier
        
        total_options_pnl = 0.0
        individual_pnls = []
        
        # Calculate PnL for each option
        for idx, (_, row) in enumerate(eth_options.iterrows()):
            strike = row['Strike']
            size = row['Size']
            time_to_expiry = row['Time_to_Expiry']
            option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
            volatility = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
            
            # Clean option type
            if 'call' in option_type:
                option_type = 'call'
            elif 'put' in option_type:
                option_type = 'put'
            else:
                option_type = 'call'
            
            # Current and stressed option prices
            current_option_price = black_scholes_premium(current_eth_price, strike, time_to_expiry, 0.0, volatility, option_type)
            stressed_option_price = black_scholes_premium(stressed_eth_price, strike, time_to_expiry, 0.0, volatility, option_type)
            
            # Position PnL
            position_pnl = size * (stressed_option_price - current_option_price)
            total_options_pnl += position_pnl
            individual_pnls.append(f"#{idx+1}: ${position_pnl:,.0f}")
        
        # Format individual contributions
        contributions = " | ".join(individual_pnls)
        if len(contributions) > 50:
            contributions = contributions[:47] + "..."
        
        print(f"{pct_change:>+3d}%       ${stressed_eth_price:>9,.0f}   ${total_options_pnl:>17,.0f}   {contributions}")
    
    print("\n")
    
    # Additional summary statistics
    print("=" * 80)
    print("ETH OPTIONS RISK METRICS")
    print("=" * 80)
    
    # Calculate some key risk metrics
    pnl_5pct_down = 0.0
    pnl_5pct_up = 0.0
    pnl_10pct_down = 0.0
    pnl_10pct_up = 0.0
    
    for _, row in eth_options.iterrows():
        strike = row['Strike']
        size = row['Size']
        time_to_expiry = row['Time_to_Expiry']
        option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
        volatility = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
        
        if 'call' in option_type:
            option_type = 'call'
        elif 'put' in option_type:
            option_type = 'put'
        else:
            option_type = 'call'
        
        current_option_price = black_scholes_premium(current_eth_price, strike, time_to_expiry, 0.0, volatility, option_type)
        
        # 5% moves
        price_5pct_down = current_eth_price * 0.95
        price_5pct_up = current_eth_price * 1.05
        option_price_5pct_down = black_scholes_premium(price_5pct_down, strike, time_to_expiry, 0.0, volatility, option_type)
        option_price_5pct_up = black_scholes_premium(price_5pct_up, strike, time_to_expiry, 0.0, volatility, option_type)
        
        pnl_5pct_down += size * (option_price_5pct_down - current_option_price)
        pnl_5pct_up += size * (option_price_5pct_up - current_option_price)
        
        # 10% moves
        price_10pct_down = current_eth_price * 0.90
        price_10pct_up = current_eth_price * 1.10
        option_price_10pct_down = black_scholes_premium(price_10pct_down, strike, time_to_expiry, 0.0, volatility, option_type)
        option_price_10pct_up = black_scholes_premium(price_10pct_up, strike, time_to_expiry, 0.0, volatility, option_type)
        
        pnl_10pct_down += size * (option_price_10pct_down - current_option_price)
        pnl_10pct_up += size * (option_price_10pct_up - current_option_price)
    
    print(f"ETH Options PnL at 5% ETH price decline:  ${pnl_5pct_down:>12,.0f}")
    print(f"ETH Options PnL at 5% ETH price increase: ${pnl_5pct_up:>12,.0f}")
    print(f"ETH Options PnL at 10% ETH price decline: ${pnl_10pct_down:>12,.0f}")
    print(f"ETH Options PnL at 10% ETH price increase: ${pnl_10pct_up:>12,.0f}")
    print()
    
else:
    print("No ETH options found for debugging.")

ETH OPTIONS DETAILED DEBUG - PnL ANALYSIS
Current ETH Price: $4,404.07
Number of ETH Options: 2


--- ETH OPTION #1 ---
Strike: $3,800.00
Size: 200
Expiry: 26Sep25 09:00
Time to Expiry: 0.0492 years
Type: PUT
Volatility: 71.10%
Current Premium per Contract: $60.65
Current Total Position Value: $12,130 ($60.65 × 200)

Price Move   ETH Price    Premium      Total Value     Position PnL   
(%)          ($)          ($)          (Premium*Size)  ($)            
--------------------------------------------------------------------------------
-10%       $    3,964   $    170.91   $       34,181   $       22,051
 -9%       $    4,008   $    155.42   $       31,085   $       18,954
 -8%       $    4,052   $    141.06   $       28,213   $       16,082
 -7%       $    4,096   $    127.78   $       25,556   $       13,425
 -6%       $    4,140   $    115.52   $       23,104   $       10,974
 -5%       $    4,184   $    104.24   $       20,848   $        8,717
 -4%       $    4,228   $     93.88   

In [10]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def calculate_token_pnl(token, multiplier):
    """
    Calculate PnL for a specific token at a given price multiplier
    """
    token_data = df[df['Token'] == token]
    if len(token_data) == 0:
        return 0.0
        
    current_price = token_data['Current_Price'].iloc[0]
    stressed_price = current_price * multiplier
    price_change = stressed_price - current_price
    
    token_pnl = 0.0
    
    # Spot and Future PnL (linear) - use adjusted delta exposure
    spot_future_data = token_data[token_data['Position_Type'].str.upper().isin(['SPOT', 'FUTURE'])]
    if len(spot_future_data) > 0:
        total_delta_exposure = spot_future_data['Delta_Exposure'].sum()
        spot_future_pnl = total_delta_exposure * price_change
        token_pnl += spot_future_pnl
    
    # Options PnL (non-linear)
    option_data = token_data[token_data['Position_Type'].str.upper() == 'OPTION']
    if len(option_data) > 0:
        for _, row in option_data.iterrows():
            size = row['Size']
            
            # Current option price
            S_current = current_price
            K = row['Strike']
            T = row['Time_to_Expiry']
            r = 0.0
            sigma = row['Volatility'] if pd.notna(row['Volatility']) and row['Volatility'] > 0 else 0.5
            option_type = str(row['Option_Type']).lower() if pd.notna(row['Option_Type']) else 'call'
            
            if 'call' in option_type:
                option_type = 'call'
            elif 'put' in option_type:
                option_type = 'put'
            else:
                option_type = 'call'
            
            if pd.notna(S_current) and pd.notna(K) and S_current > 0 and K > 0:
                current_option_price = black_scholes_premium(S_current, K, T, r, sigma, option_type)
                stressed_option_price = black_scholes_premium(stressed_price, K, T, r, sigma, option_type)
                
                option_pnl = size * (stressed_option_price - current_option_price)
                token_pnl += option_pnl
    
    return token_pnl

def calculate_portfolio_pnl(multiplier):
    """Calculate total portfolio PnL for a given price multiplier"""
    total_pnl = 0.0
    for token in df['Token'].unique():
        token_pnl = calculate_token_pnl(token, multiplier)
        total_pnl += token_pnl
    return total_pnl

# Find top 7 PnL contributors at 5% move
print("Finding top PnL contributors...")
token_pnl_at_5pct = {}
for token in df['Token'].unique():
    pnl_up = abs(calculate_token_pnl(token, 1.05))
    pnl_down = abs(calculate_token_pnl(token, 0.95))
    max_pnl = max(pnl_up, pnl_down)
    token_pnl_at_5pct[token] = max_pnl

# Get top 7 tokens by max PnL impact
top_tokens = sorted(token_pnl_at_5pct.items(), key=lambda x: x[1], reverse=True)[:7]
top_token_names = [token for token, _ in top_tokens]

print(f"Top 7 PnL contributors: {top_token_names}")

# Get current BTC and ETH prices for reference
btc_price = df[df['Token'] == 'BTC']['Current_Price'].iloc[0] if 'BTC' in df['Token'].values else None
eth_price = df[df['Token'] == 'ETH']['Current_Price'].iloc[0] if 'ETH' in df['Token'].values else None

print(f"Current BTC Price: ${btc_price:,.0f}" if btc_price else "BTC price not found")
print(f"Current ETH Price: ${eth_price:,.0f}" if eth_price else "ETH price not found")

# Generate PnL curves
print("Generating PnL curves...")
price_moves = list(range(-10, 11, 1))  # -10% to +10% in 1% steps

# Colors for different tokens
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Create interactive Plotly visualization
fig = go.Figure()

# Add individual token PnL curves
for i, token in enumerate(top_token_names):
    token_pnl_values = []
    for pct_change in price_moves:
        multiplier = 1 + (pct_change / 100)
        pnl = calculate_token_pnl(token, multiplier)
        token_pnl_values.append(pnl)
    
    fig.add_trace(go.Scatter(
        x=price_moves,
        y=token_pnl_values,
        mode='lines+markers',
        name=f'{token}',
        line=dict(color=colors[i], width=2),
        marker=dict(size=6, color=colors[i]),
        hovertemplate=f'{token}<br>Price Move: %{{x}}%<br>PnL: $%{{y:,.0f}}<extra></extra>'
    ))

# Add total portfolio PnL curve (thicker line)
portfolio_pnl_values = []
for pct_change in price_moves:
    multiplier = 1 + (pct_change / 100)
    pnl = calculate_portfolio_pnl(multiplier)
    portfolio_pnl_values.append(pnl)
    print(f"{pct_change:+2d}%: ${pnl:>12,.0f}")

fig.add_trace(go.Scatter(
    x=price_moves,
    y=portfolio_pnl_values,
    mode='lines+markers',
    name='Total Portfolio',
    line=dict(color='black', width=4),
    marker=dict(size=8, color='black'),
    hovertemplate='Total Portfolio<br>Price Move: %{x}%<br>PnL: $%{y:,.0f}<extra></extra>'
))

# Add reference lines
fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=0, line_dash="dash", line_color="gray", opacity=0.5)

# Get the y-axis range to position price references below the chart
y_min = min(portfolio_pnl_values + [pnl for token_pnl in [calculate_token_pnl(token, 0.9) for token in top_token_names] for pnl in [token_pnl]])
y_max = max(portfolio_pnl_values + [pnl for token_pnl in [calculate_token_pnl(token, 1.1) for token in top_token_names] for pnl in [token_pnl]])
y_range = y_max - y_min
reference_y_position = y_min - (y_range * 0.1)  # Position 10% below the lowest point

# Add BTC and ETH price references at 1% intervals
if btc_price and eth_price:
    for pct_change in price_moves:
        multiplier = 1 + (pct_change / 100)
        btc_ref_price = btc_price * multiplier
        eth_ref_price = eth_price * multiplier
        
        # Add annotations for BTC and ETH prices
        fig.add_annotation(
            x=pct_change,
            y=reference_y_position,
            text=f"BTC: ${btc_ref_price:,.0f}<br>ETH: ${eth_ref_price:,.0f}",
            showarrow=False,
            font=dict(size=8, color="darkblue"),
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="lightblue",
            borderwidth=1,
            xanchor="center",
            yanchor="top"
        )

# Update layout
fig.update_layout(
    title={
        'text': 'Portfolio PnL vs Price Movement (Top Contributors)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18}
    },
    xaxis_title='Price Movement (%)',
    yaxis_title='Portfolio PnL ($)',
    xaxis=dict(
        tickmode='linear',
        tick0=-10,
        dtick=1,
        gridcolor='lightgray',
        gridwidth=1,
        range=[-10.5, 10.5]  # Slightly extend range for better visibility
    ),
    yaxis=dict(
        tickformat='$,.0f',
        gridcolor='lightgray',
        gridwidth=1,
        range=[reference_y_position - (y_range * 0.05), y_max + (y_range * 0.1)]  # Extend range to show price references
    ),
    hovermode='x unified',
    width=1400,
    height=800,
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Show the interactive plot
fig.show()

# Print summary statistics
print(f"\nPnL Summary:")
print(f"Maximum Loss (10% down): ${min(portfolio_pnl_values):,.0f}")
print(f"Maximum Gain (10% up):   ${max(portfolio_pnl_values):,.0f}")
print(f"PnL at 0% move:         ${portfolio_pnl_values[10]:,.0f}")  # Index 10 is 0% in the -10 to +10 range

Finding top PnL contributors...
Top 7 PnL contributors: ['ETH', 'ICP', 'SOL', 'BTC', 'OXT', 'XRP', 'NEAR']
Current BTC Price: $112,195
Current ETH Price: $4,404
Generating PnL curves...
-10%: $      35,702
-9%: $      24,757
-8%: $      15,265
-7%: $       7,237
-6%: $         682
-5%: $      -3,221
-4%: $      -5,521
-3%: $      -6,344
-2%: $      -5,694
-1%: $      -3,577
+0%: $           0
+1%: $       5,028
+2%: $      11,497
+3%: $      19,398
+4%: $      28,719
+5%: $      39,448
+6%: $      51,572
+7%: $      65,078
+8%: $      79,949
+9%: $      96,168
+10%: $     113,715



PnL Summary:
Maximum Loss (10% down): $-6,344
Maximum Gain (10% up):   $113,715
PnL at 0% move:         $0
